In [22]:
print("OK!")

OK!


In [125]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [126]:
PINECONE_API_KEY = "744c149b-f5d1-4817-be42-1eacb06e6e0e"
PINECONE_API_ENV = "quickstart"

In [132]:
pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [138]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="744c149b-f5d1-4817-be42-1eacb06e6e0e")

pc.create_index(
  name="less-index",
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)


In [139]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [140]:
extracted_data = load_pdf("/Users/harshjain/Desktop/End-to-end-Medical-Chatbot-using-Llama2/data")

In [141]:
# extracted_data

In [142]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [143]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [144]:
# text_chunks

In [145]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [146]:
embeddings = download_hugging_face_embeddings()

In [147]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [148]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [106]:
# query_result

In [164]:
import os
from pinecone import Pinecone, ServerlessSpec

# Assign the API key to a variable
PINECONE_API_KEY = "744c149b-f5d1-4817-be42-1eacb06e6e0e"

# Initialize the Pinecone client using the API key
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "less-index"

# Create the index with a serverless configuration
pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# Access the index
index = pc.Index(index_name)

# Add embeddings to the index
for t in text_chunks:
    index.upsert(item_ids=[t.id], vectors=[t.page_content])

# Retrieve embeddings from the index
# (You can customize this part according to your use case)
query_vector = your_query_vector_here
results = index.query(queries=query_vector, top_k=10)

# Process the search results
for result in results:
    print(result)


ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '7e9a001ac12b3d1635c30b73745f42fe', 'Date': 'Tue, 04 Jun 2024 21:51:57 GMT', 'Server': 'Google Frontend', 'Content-Length': '196', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Request failed. You've reached the max serverless indexes allowed in project Chatbot (5). To add more serverless indexes, upgrade your plan."},"status":403}


In [73]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name , embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])